In [3]:
from Automata_lib import Automaton, OutputMode
import re

In [4]:
a = Automaton("/root/src/bakalarka/Automata/Automata_lib/Examples/non_mon.txt")

In [23]:
a = Automaton("/root/src/bakalarka/Automata/Automata_lib/Examples/mono.txt")

In [15]:
a = Automaton("/root/src/bakalarka/Automata/Automata_lib/Examples/example0_without_MVL.txt")

In [10]:
def is_monotonic(a: Automaton):
    digrhaph, rewrites = to_digraph(a)
    print(digrhaph)
    print(rewrites)
    to_check= []
    for i in rewrites.keys():
        to_check.append(i)
    
    #check if nondeterministic could skipp rewrite and rewrite later
    reversed_digraph = defaultdict(list)
    for key in digrhaph.keys():
        for item in digrhaph[key]:
            reversed_digraph[item].append(key) 
            reversed_digraph[key] 

    used = {key: False for key in digrhaph.keys()}
    for state in to_check:
        st = [state]
        distance = {i:0 if i == state else -1 for i in reversed_digraph.keys()}
        while st:
            current = st.pop(0)
            if distance[current] >= a.size_of_window and current in rewrites.keys():
                return False
            for next_state in reversed_digraph[current]:
                if distance[current] == float("inf"):
                    if not distance[next_state] == float("inf"):
                        distance[next_state] = float("inf")
                    else:
                        continue # don't loop in infinity
                if distance[next_state] == -1:
                    distance[next_state] = distance[current] + 1
                else:
                    distance[next_state] = float("inf")
                st.append(next_state)

                
    # while to_compute:
    #     current = to_compute.pop(0)
    #     for state in reversed_digraph[current]:
    #         if distance[state] == -1:
    #             distance[state] = distance[current] + 1
    #             to_compute.append(state)

    # for state in digrhaph.keys():
    #     if state in rewrites.keys():
    #         difference = digrhaph[state] - rewrites[state]["to_state"]
    #         for state in difference:
    #             find_rewrite_after_n_steps(digrhaph, rewrites, diff, a.size_of_window)

    #checking if in next run there could be rewrite in k-1-(|word|-|new_word|)
    to_check_without_states = [i[2:] for i in to_check]
    for i in to_check:
        for rewrite in rewrites[i]:
            rewrite = rewrite["rewriten_to"]
            print(rewrite)
            for possible_bad_state in generate_pre_complete_word(a,a.size_of_window,rewrite): #k-1 
                if str(possible_bad_state) in to_check_without_states:
                    print(possible_bad_state)
                    return False
    return True

In [26]:
is_monotonic(a)

Initial -> "q0['#', 'd', '$']"
Initial -> "q0['#', 'a', 'c', 'b']"
Initial -> "q0['#', 'e', '$']"
Initial -> "q0['#', 'a', 'a', 'a']"
Initial -> "q0['#', 'a', 'a', 'c']"
"q0['#', 'd', '$']" -> Accept
"q0['#', 'e', '$']" -> Accept
"q0['#', 'a', 'a', 'a']" -> "q0['a', 'a', 'a', 'a']"
"q0['#', 'a', 'a', 'a']" -> "q0['a', 'a', 'a', 'c']"
"q0['#', 'a', 'a', 'c']" -> "q0['a', 'a', 'c', 'b']"
"q0['a', 'a', 'a', 'a']" -> "q0['a', 'a', 'a', 'a']"
"q0['a', 'a', 'a', 'a']" -> "q0['a', 'a', 'a', 'c']"
"q0['a', 'a', 'a', 'c']" -> "q0['a', 'a', 'c', 'b']"
"q0['a', 'a', 'c', 'b']" -> "qe['b', 'c', 'b', 'c']"
"q0['a', 'a', 'c', 'b']" -> "qe['b', 'c', 'b', 'e']"
"q0['a', 'a', 'c', 'b']" -> "qe['c', 'b', 'c', 'b']"
"q0['a', 'a', 'c', 'b']" -> "qe['e', '$']"
"qe['c', 'b', 'c', 'b']" -> "qe['b', 'c', 'b', 'c']"
"qe['c', 'b', 'c', 'b']" -> "qe['b', 'c', 'b', 'e']"
"qe['b', 'c', 'b', 'c']" -> "qe['c', 'b', 'c', 'b']"
"qe['b', 'c', 'b', 'e']" -> Restart
"qe['e', '$']" -> Restart
"qd['c', 'b', 'c', 'b']" -> "

False

In [6]:
def generate_pre_complete_word(a:Automaton, new_lenght:int, content: list): # [ ? ? a b] 
    ret = []
    l = a.alphabet + a.working_alphabet
    if len(content) +1 == new_lenght: l+= ["#"]
    if new_lenght < len(content):
        return []
    if new_lenght == len(content):
        return [content]
    for i in l:
        new_word = content.copy()
        new_word = [i] + new_word
        ret.extend(generate_pre_complete_word(a,new_lenght,new_word))
    return ret

In [7]:
#TODO should be done with generator to ensure space complexity
def generate_post_complete_word(a:Automaton, new_lenght:int, content: list): # [ ? ? a b] 
    ret = []
    l = a.alphabet + a.working_alphabet +["$"]
    if new_lenght < len(content):
        return []
    if content and (content[-1] == "$" or new_lenght == len(content)):
        return [content]
    for i in l:
        new_word = content.copy()
        new_word += i
        ret.extend(generate_post_complete_word(a,new_lenght,new_word))
    return ret

    

In [25]:
from collections import defaultdict
def to_digraph(a: Automaton):
    possible_states = set(["Accept", "Restart"])
    for key in a.instructions.keys():
        for window in a.instructions[key]:
            possible_states.add(key + window) #beware of star
    # print(possible_states)
    possible_states = list(possible_states)
    digraph = defaultdict(list)
    rewrites = defaultdict(list)
    for state in possible_states:
        if "#" in state:
            print("Initial -> " + f'"{state}"')
            digraph["Initial"].append(state)
    for key in a.instructions.keys():
        for window in a.instructions[key]:
            for instruction in a.instructions[key][window]:
                if type(instruction) is str:
                    print(f'"{key+window}"' + " -> " + instruction)
                    digraph[key+window].append(instruction)
                elif type(instruction) is list and len(instruction) == 2:
                    to_state = instruction[0]
                    instruction = instruction[1]
                    if instruction == "MVR": 
                        for c in a.alphabet + a.working_alphabet + ["$"]:
                            new_window = str(eval(window)[1:] + [c])
                            if to_state+new_window in possible_states + [to_state+"['*']"]:
                                print(f'"{key+window}" -> "{to_state+new_window}"')
                                digraph[key+window].append(to_state+new_window)
                    elif instruction == "MVL":
                        print("TODO MVL")
                    elif re.match(r"^\[.*\]$", instruction):
                        rewrite_to = eval(instruction)  # making array from string
                        if to_state+"['*']" in possible_states:
                            tt = to_state+"['*']"
                            rewrites[key+window].append({"rewriten_to":rewrite_to,"to_state": tt})
                            print(f'"{key+window}" -> "{tt}"')
                            digraph[key+window].append(tt)
                            # continue # the following code is some what hard to compute but probably needed
                        for possible_window in map(str,generate_post_complete_word(a,a.size_of_window,rewrite_to)): #TODO definice .... 
                            if to_state+possible_window in possible_states:
                                print(f'"{key+window}" -> "{to_state+possible_window}"')
                                digraph[key+window].append(to_state+possible_window)
                                rewrites[key+window].append({"rewriten_to":rewrite_to, "to_state":to_state+possible_window})

    return digraph,rewrites

In [ ]:
generate_post_complete_word(a,3,["b"])

In [24]:
to_digraph(a)

Initial -> "q0['#', 'd', '$']"
Initial -> "q0['#', 'a', 'c', 'b']"
Initial -> "q0['#', 'e', '$']"
Initial -> "q0['#', 'a', 'a', 'a']"
Initial -> "q0['#', 'a', 'a', 'c']"
"q0['#', 'd', '$']" -> Accept
"q0['#', 'e', '$']" -> Accept
"q0['#', 'a', 'a', 'a']" -> "q0['a', 'a', 'a', 'a']"
"q0['#', 'a', 'a', 'a']" -> "q0['a', 'a', 'a', 'c']"
"q0['#', 'a', 'a', 'c']" -> "q0['a', 'a', 'c', 'b']"
"q0['a', 'a', 'a', 'a']" -> "q0['a', 'a', 'a', 'a']"
"q0['a', 'a', 'a', 'a']" -> "q0['a', 'a', 'a', 'c']"
"q0['a', 'a', 'a', 'c']" -> "q0['a', 'a', 'c', 'b']"
"q0['a', 'a', 'c', 'b']" -> "qe['b', 'c', 'b', 'c']"
"q0['a', 'a', 'c', 'b']" -> "qe['b', 'c', 'b', 'e']"
"q0['a', 'a', 'c', 'b']" -> "qe['c', 'b', 'c', 'b']"
"q0['a', 'a', 'c', 'b']" -> "qe['e', '$']"
"qe['c', 'b', 'c', 'b']" -> "qe['b', 'c', 'b', 'c']"
"qe['c', 'b', 'c', 'b']" -> "qe['b', 'c', 'b', 'e']"
"qe['b', 'c', 'b', 'c']" -> "qe['c', 'b', 'c', 'b']"
"qe['b', 'c', 'b', 'e']" -> Restart
"qe['e', '$']" -> Restart
"qd['c', 'b', 'c', 'b']" -> "

(defaultdict(list,
             {'Initial': ["q0['#', 'd', '$']",
               "q0['#', 'a', 'c', 'b']",
               "q0['#', 'e', '$']",
               "q0['#', 'a', 'a', 'a']",
               "q0['#', 'a', 'a', 'c']"],
              "q0['#', 'd', '$']": ['Accept'],
              "q0['#', 'e', '$']": ['Accept'],
              "q0['#', 'a', 'a', 'a']": ["q0['a', 'a', 'a', 'a']",
               "q0['a', 'a', 'a', 'c']"],
              "q0['#', 'a', 'a', 'c']": ["q0['a', 'a', 'c', 'b']"],
              "q0['a', 'a', 'a', 'a']": ["q0['a', 'a', 'a', 'a']",
               "q0['a', 'a', 'a', 'c']"],
              "q0['a', 'a', 'a', 'c']": ["q0['a', 'a', 'c', 'b']"],
              "q0['a', 'a', 'c', 'b']": ["qe['b', 'c', 'b', 'c']",
               "qe['b', 'c', 'b', 'e']",
               "qe['c', 'b', 'c', 'b']",
               "qe['e', '$']"],
              "qe['c', 'b', 'c', 'b']": ["qe['b', 'c', 'b', 'c']",
               "qe['b', 'c', 'b', 'e']"],
              "qe['b', 'c', 'b', 'c'

In [17]:
a.instructions

{'q0': {"['a', 'a', 'a']": [['q0', 'MVR']],
  "['a', 'a', 'b']": [['q0', 'MVR']],
  "['a', 'b', 'b']": [['q0', 'MVR']],
  "['a', 'b', 'c']": [['q0', 'MVR']],
  "['b', 'b', 'b']": [['q0', 'MVR']],
  "['b', 'b', 'c']": [['q0', 'MVR'], ['qr', "['C', 'c']"]],
  "['#', 'c', '$']": ['Accept'],
  "['#', 'd', '$']": ['Accept'],
  "['#', 'a', 'b']": [['q0', 'MVR']],
  "['#', 'a', 'a']": [['q0', 'MVR']],
  "['b', 'b', 'd']": [['qr', "['D', 'd']"]],
  "['b', 'b', 'D']": [['qr', "['D', 'D']"]],
  "['b', 'b', 'C']": [['qr', "['C', 'C']"]],
  "['a', 'b', 'C']": [['qr', "['C']"]],
  "['a', 'a', 'C']": [['qr', '[]']],
  "['#', 'a', 'C']": [['qr', "['#']"]],
  "['a', 'a', 'D']": [['qr', "['a']"]],
  "['#', 'a', 'D']": [['qr', "['#']"]]},
 'qr': {"['*']": ['Restart']}}

In [ ]:
def can_restart_with_window(state, window):
    pass

In [55]:
def can_restart(state):
    s = [state]
    visited = [state]
    possible_states = []
    while s:
        cur_state = s.pop()
        for window in a.instructions[cur_state]:
            for right_side in a.instructions[cur_state][window]:
                if right_side == "Accept": continue
                if right_side == "Restart":
                    return True
                if right_side[0] not in visited:
                    possible_states.append(right_side[0])
                    visited.append(right_side[0])
                    s.append(right_side[0])
    return False

In [56]:
# a.instructions
can_restart("q0")

True

In [8]:
a.evaluate("#aabbc$")

['#', 'a', 'a', 'b', 'b', 'c', '$']


KeyError: None

In [15]:
a.instructions

{'q0': {"['a', 'a', 'a']": [['q0', 'MVR']],
  "['a', 'a', 'b']": [['q0', 'MVR']],
  "['a', 'b', 'b']": [['q0', 'MVR']],
  "['a', 'b', 'c']": [['q0', 'MVR']],
  "['b', 'b', 'b']": [['q0', 'MVR']],
  "['b', 'b', 'c']": [['q0', 'MVR']],
  "['b', 'b', 'd']": [['q0', 'MVR']],
  "['#', 'c', '$']": ['Accept'],
  "['#', 'd', '$']": ['Accept'],
  "['#', 'a', 'b']": [['q0', 'MVR']],
  "['#', 'a', 'a']": [['q0', 'MVR']],
  "['b', 'c', '$']": [['qc', 'MVL']],
  "['b', 'd', '$']": [['qd', 'MVL']]},
 'qr': {"['*']": ['Restart']},
 'qc': {"['a', 'b', 'c']": [['qr', "['c']"]],
  "['b', 'b', 'c']": [['qc', 'MVL']],
  "['b', 'b', 'b']": [['qc', 'MVL']],
  "['a', 'b', 'b']": [['qr', "['b']"]]},
 'qd': {"['b', 'b', 'd']": [['qd', 'MVL']],
  "['b', 'b', 'b']": [['qd', 'MVL']],
  "['a', 'b', 'b']": [['qr', '[]']]}}

In [14]:
a.starting_position = 0
a.size_of_window = 3
a.initial_state = "q0"
a.add_to_alphabet("a", "b", "c", "d")

for i in ["aaa", "aab", "abb", "abc", "bbb", "bbc", "bbd"]:
    a.add_instr("q0", i, "q0", "MVR")

a.add_one_instr("q0", "#c$", "Accept")
a.add_one_instr("q0", "#d$", "Accept")

a.add_instr("q0", "#ab", "q0", "MVR")
a.add_instr("q0", "#aa", "q0", "MVR")
a.add_instr("q0", "bc$", "qc", "MVL")
a.add_instr("q0", "bd$", "qd", "MVL")

a.add_one_instr("qr", "*", "Restart")

a.add_instr("qc", "abc", "qr", "c")
a.add_instr("qc", "bbc", "qc", "MVL")
a.add_instr("qc", "bbb", "qc", "MVL")
a.add_instr("qc", "abb", "qr", "b")
a.add_instr("qd", "bbd", "qd", "MVL")
a.add_instr("qd", "bbb", "qd", "MVL")
a.add_instr("qd", "abb", "qr", "[]")

In [16]:
for i in a.instructions.keys():
    for w in a.instructions[i]:
        print(a.instructions[i][w])

[['q0', 'MVR']]
[['q0', 'MVR']]
[['q0', 'MVR']]
[['q0', 'MVR']]
[['q0', 'MVR']]
[['q0', 'MVR']]
[['q0', 'MVR']]
['Accept']
['Accept']
[['q0', 'MVR']]
[['q0', 'MVR']]
[['qc', 'MVL']]
[['qd', 'MVL']]
['Restart']
[['qr', "['c']"]]
[['qc', 'MVL']]
[['qc', 'MVL']]
[['qr', "['b']"]]
[['qd', 'MVL']]
[['qd', 'MVL']]
[['qr', '[]']]


In [17]:
a.evaluate("#aaabbbc$")

['#', 'a', 'a', 'a', 'b', 'b', 'b', 'c', '$']
remaining tuples = []
number of copies of text = 4
(#, a, a), a, b, b, b, c, $ | state: q0
#, (a, a, a), b, b, b, c, $ | state: q0
#, a, (a, a, b), b, b, c, $ | state: q0
#, a, a, (a, b, b), b, c, $ | state: q0
#, a, a, a, (b, b, b), c, $ | state: q0
#, a, a, a, b, (b, b, c), $ | state: q0
#, a, a, a, b, b, (b, c, $) | state: q0
#, a, a, a, b, (b, b, c), $ | state: qc
#, a, a, a, (b, b, b), c, $ | state: qc
#, a, a, (a, b, b), b, c, $ | state: qc
#, a, a, (b, b, c), $ | state: qr | ['a', 'b', 'b'] --> ['b']
(#, a, a), b, b, c, $ | state: q0
#, (a, a, b), b, c, $ | state: q0
#, a, (a, b, b), c, $ | state: q0
#, a, a, (b, b, c), $ | state: q0
#, a, a, b, (b, c, $) | state: q0
#, a, a, (b, b, c), $ | state: qc
#, a, (a, b, b), c, $ | state: qc
#, a, (b, c, $) | state: qr | ['a', 'b', 'b'] --> ['b']
(#, a, b), c, $ | state: q0
#, (a, b, c), $ | state: q0
#, a, (b, c, $) | state: q0
#, (a, b, c), $ | state: qc
#, (c, $) | state: qr | ['a', 'b', 

True

In [5]:
a.instructions

{'q0': {"['a', 'a', 'a']": [['q0', 'MVR']],
  "['a', 'a', 'b']": [['q0', 'MVR']],
  "['a', 'b', 'b']": [['q0', 'MVR']],
  "['a', 'b', 'c']": [['q0', 'MVR']],
  "['b', 'b', 'b']": [['q0', 'MVR']],
  "['b', 'b', 'c']": [['q0', 'MVR']],
  "['b', 'b', 'd']": [['q0', 'MVR']],
  "['#', 'c', '$']": ['Accept'],
  "['#', 'd', '$']": ['Accept'],
  "['#', 'a', 'b']": [['q0', 'MVR']],
  "['#', 'a', 'a']": [['q0', 'MVR']],
  "['b', 'c', '$']": [['qc', 'MVL']],
  "['b', 'd', '$']": [['qd', 'MVL']]},
 'qr': {"['*']": ['Restart']},
 'qc': {"['a', 'b', 'c']": [['qr', "['c']"]],
  "['b', 'b', 'c']": [['qc', 'MVL']],
  "['b', 'b', 'b']": [['qc', 'MVL']],
  "['a', 'b', 'b']": [['qr', "['b']"]]},
 'qd': {"['b', 'b', 'd']": [['qd', 'MVL']],
  "['b', 'b', 'b']": [['qd', 'MVL']],
  "['a', 'b', 'b']": [['qr', '[]']]}}

In [6]:
a = Automaton(file="../0n1n.txt")

In [11]:
a.evaluate("#000111$")

['#', '0', '0', '0', '1', '1', '1', '$']
remaining tuples = []
number of copies of text = 3
(#, 0, 0, 0), 1, 1, 1, $ | state: q0
#, (0, 0, 0, 1), 1, 1, $ | state: q0
#, 0, (0, 0, 1, 1), 1, $ | state: q0
#, 0, (0, 1, 1, $) | state: qr | ['0', '0', '1', '1'] --> ['0', '1']
(#, 0, 0, 1), 1, $ | state: q0
#, (0, 0, 1, 1), $ | state: q0
#, (0, 1, $) | state: qr | ['0', '0', '1', '1'] --> ['0', '1']
(#, 0, 1, $) | state: q0


True

In [10]:
a.logs

"['#', '0', '0', '0', '1', '1', '1', '$']\nremaining tuples = []\nnumber of copies of text = 3\n(#, 0, 0, 0), 1, 1, 1, $ | state: q0\n#, (0, 0, 0, 1), 1, 1, $ | state: q0\n#, 0, (0, 0, 1, 1), 1, $ | state: q0\n#, 0, (0, 1, 1, $) | state: qr | ['0', '0', '1', '1'] --> ['0', '1']\n(#, 0, 0, 1), 1, $ | state: q0\n#, (0, 0, 1, 1), $ | state: q0\n#, (0, 1, $) | state: qr | ['0', '0', '1', '1'] --> ['0', '1']\n(#, 0, 1, $) | state: q0\n['#', '0', '0', '0', '1', '1', '1', '$']\nremaining tuples = []\nnumber of copies of text = 3\n(#, 0, 0, 1), 1, $ | state: q0\n(#, 0, 1, $) | state: q0\n"